
Vedlegg tilgjengelige fram til 26. nov. 2019
 
Hei!

Nå er jeg ikke så veldig langt unna med å ha ferdig analysen av de forrige resultatene jeg fikk. Det er ganske gøy å holde på med det. Men jeg jobber på spreng for å holde tidsskjemaet mitt :)  


Dette betyr at jeg snart gjerne vil få hentet ut et siste sett med resultater. Som vi snakket om sist, da ønsker jeg å gjøre bruk av flere automatisk genererte temaer. Jeg satser på å sende deg nye temaer innen torsdag. Det vil bli mindre omfattende enn sist. 



Men kunne du i mellomtiden hjelpe meg litt med “dyp-topic-modellering” og legge inn flere måter å generere temaer på i notebøkene av de kvinnelige forfatterne mine? 


Hertha- notebooken er genial i så måte. Her har du lagt inn mange forskjellige parametere for tema-generering.   
Klikk for å laste ned
TM_Hertha.ipynb
385 kB


Kunne du sett litt nærmere på de andre notebookene også? Er det flere nyttige fremgangsmåter for automatisk tema-generering du kan legge til? 

Klikk for å laste ned
TM_Til Sverriges Qvinder .ipynb
13 kB
 
Klikk for å laste ned
denconstitutionelle.txt
14 kB
 
Klikk for å laste ned
Amtmandens døtre .ipynb
1,4 MB
 
Klikk for å laste ned
Indiana.ipynb
3,1 MB
 
Klikk for å laste ned
sand-indiana.txt
570 kB


I Amtmandens Døtre-notebooken har vi brukt denne teksten  
Klikk for å laste ned
ad3.txt
667 kB
 men det ville vært veldig fint om vi heller kunne brukt 1855-utgaven: 
https://www.nb.no/items/fbf5c6432c86426817fe167282f603ad?page=519&searchText=Amtmandens%20døttre


Til slutt skulle jeg veldig gjerne laget meg et dokument av de tre artiklene til Aasta Hansteen også. For jeg vil gjerne også få generert automatiske temaer fra hennes tekster. Er det mulig å ta ut Dagbladet-utgavene og legge den inn i et txt-dokument? Jeg kan i tilfelle gå inn i det etterpå og fjerne all annen tekst fra de respektive avisene (slik jeg har gjort for “Til Sverriges Qvinder”) . Her er de tre Dagblad-utgavene det gjelder: 

Klikk for å laste ned
Hansteen 12 August 1870.pdf
19,9 MB
 
Klikk for å laste ned
Hansteen 06 August 1870.pdf
21,4 MB
 
Klikk for å laste ned
Hansteen 13 July 1870.pdf
20,6 MB


In [ ]:
import requests
import os
from IPython.display import HTML, Markdown, display
from urllib.parse import urlparse, urljoin
import ipywidgets as widgets

def code_toggle(button_text = "Klikk for å vise/skjule kodeceller"):
    from IPython.display import HTML, display

    display(
        HTML(
        '''<div>
                <style>
                 .mybutton {
                    background-color: lightgrey;
                    border: none;
                    color: white;
                    padding: 10px 16px;
                    text-align: center;
                    text-decoration: none;
                    display: inline-block;
                    font-size: 16px;
                    margin: 4px 2px;
                    cursor: pointer;
                }
            </style>
            <script>
                code_show=true; 
                function code_toggle() {
                 if (code_show){
                 $('div.input').hide();
                 } else {
                 $('div.input').show();
                 }
                 code_show = !code_show
                } 
                $( document ).ready(code_toggle);
            </script>
            <form  action="javascript:code_toggle()">
                <input class='mybutton' type="submit" value=''' + '"'  + button_text + '"' + '''>
            </form>
        </div>'''
        ))
    
def printmd(S):
    display(Markdown(S))
    return

def update(module="", overwrite=True, silent=False):
    """Fetch modules from Github and write them to folder"""
    nba = requests.get(
        "https://raw.githubusercontent.com/Yoonsen/Modules/master/{module}.py".format(module=module),
        headers={'Cache-Control': 'no-cache'}
        )
    filename = '{m}.py'.format(m=module)
    if nba.status_code == 200:
        file_exists = os.path.exists(filename)
        if file_exists and not(overwrite):
            if not silent:
                printmd("File {f} exists - call `update('{m}', overwrite = True)` in order to download module `{m}` anyway".format(f = os.path.abspath(filename), m = module))
        else:
            nba = nba.text
            with open(filename,'w', encoding='UTF-8') as pyfile:
                pyfile.write(nba)
                pyfile.flush()
                pyfile.close()
            if not silent:
                printmd("Updated file `{module}.py`".format(module= os.path.abspath(module)))
    else:
        printmd( 
            """{intro} for {module} with error {code}""".format(
                intro = "An error occured during download", 
                module = module, 
                code= nba.status_code
            )
        )
    return



In [301]:
import pandas as pd
from dhlab.nbtokenizer import tokenize
from IPython.display import HTML, Markdown, display
from collections import Counter
import networkx as nx
import json

In [2]:
tokenize("hallo der borte.")

['hallo', 'der', 'borte', '.']

In [103]:
with open('ad3.txt', encoding='utf-8') as f:
    amt = f.read()

In [140]:
#temaene ble lagret med annen notebook - Heidis definisjoner_0_november
with open('../Nye tema november 2019/heidis_temadefinisjoner_november.json') as f:
    first = json.load(f)

wordbags = dict()
for f in first:
    for wb in first[f]:
        wordbags[wb] = first[f][wb]

In [125]:
def paragraphs(textfile):
    with open('ad3.txt', encoding='utf-8') as f:
        text = f.read()
    # return tokenizes paragraphs 
    return [tokenize(p) for p in text.split('\n')]

In [126]:
def paragraphs_hot(textfile, freqlim = 1):
    # return tokenizes paragraphs 
    i = 0
    struct = dict()
    for p in paragraphs(textfile):
        struct[i] = Counter(p)
        i += 1
    return pd.DataFrame(struct).fillna(0)
        

In [127]:
amth = paragraphs_hot('ad3.txt')

In [128]:
freqs = amth.sum(axis=1).sort_values()

In [50]:
def pcoll(word, paras):
    """paras as a numbered dictionary of paragraphs"""
    hits = pd.DataFrame([paras[para] for para in paras if word in paras[para]])
    return hits

In [161]:
def coll(word, paras, exp=1.2):
    if word in paras.index:
        res = (amth[[i for i in amth if amth[i][word] > 0]].sum(axis=1)**exp/freqs).sort_values(ascending=False)
    else:
        res = pd.Series()
    return res

In [284]:
def collwb(wordbag, paras, exp=1.2, column=None):
    res = pd.DataFrame()
    for w in wordbag:
        res[w] = coll(w, paras, exp)
    result = pd.DataFrame()
    if column == None:
        colname = 'coll'
    else:
        colname = column
    result[colname] = res.sum(axis=1)
    return result.sort_values(by=colname, ascending=False)

def combocoll(wordbag, paras, exps = None, freqs = None):
    
    if exps == None:
        exps = [1.05, 1.2]
    
    rescoll = pd.concat([collwb(wordbag, amth, exp = a, column = a) for a in exps], axis=1, sort=True)
    
    # wise to compute the frequencies beforehand, it takes a second or so off the time
    if freqs == None:
        freqs = paras.sum(axis=1)
    
    rescoll['freq'] = freqs[rescoll.index]
    
    return rescoll

In [238]:
def inspectcoll(collocation, freqlim = 0, colnum = None, colname = None):
    if colname == None:
        if colnum == None:
            column = collocation.columns[0]
        else:
            column = collocation.columns[colnum]
    else:
        column = colname
        
    return collocation[collocation.freq > freqlim].sort_values(by=column, ascending=False)

In [310]:
def displaycoll(collocation,n = 20, freqlim = 0):        
    for column in collocation:
        if column != 'freq':
            display(Markdown('### ' + str(column)))
            display(Markdown(', '.join(list(collocation[collocation.freq > freqlim].sort_values(by=column, ascending=False).index[:n]))))
            display(Markdown('<br>'))

In [312]:
wordbags.keys()

dict_keys(['Kvinde_', 'Sandhed_', 'Frihed_', 'Legeme_', 'Læserinde_', 'Litteratur_', 'Religion_', 'Kvindeforening_', 'Forfatterinde_', 'Myndighed_', 'Yrkesaktiv_', 'Sivil status_', 'Ligestilling_', 'Folkeoplysning_', 'Sundhed_', 'Befolkningsprosesser_'])

In [314]:
amtmand_tema = dict()
for w in wordbags.keys():
    amtmand_tema[w] = combocoll(wordbags[w], amth, exps = [1.05, 1.2, 1.5])


In [316]:
for w in amtmand_tema:
    display(Markdown("## "+ w))
    displaycoll(amtmand_tema[w], freqlim=2)

## Kvinde_

### 1.05

Kvindelighedens, ydmygende, Vist, Kvindelighed, Kvinder, Kvindes, Kavallerer, Væsener, Ballet, Damer, brogede, Indkjørselen, Vækst, romantiske, Forsagelse, Damerne, Kvindens, kvindelig, Mor, Kvinde

<br>

### 1.2

...

Lidet, Nøiagtighed, iforveien, omvendt, Mysterium, matte, Lyksalighed, Præget, hilst, udtale, Opførsel, Norsk, Scenen, ment, Mandens, nationale, Lystighed, Vished, bevidst, Beregning

<br>

### 1.5

Lidet, Opførsel, hilst, Præget, Mysterium, omvendt, Lyksalighed, Nøiagtighed, matte, iforveien, udtale, igår, Vished, Scenen, nationale, bevidst, ment, Beregning, Norsk, Mandens

<br>

In [271]:
displaycoll(coll_kvinde, n = 10, freqlim = 2)

1.02 Kvindelighedens, ydmygende, Vist, Kvindelighed, Kvinder, Kvindes, Kavallerer, Væsener, Ballet, Vækst

1.3 Mor, Moder, Damer, Dame, Damerne, Kvinde, Hustru, Kvindelighed, Kone, kvindelig

1.5 Moder, Mor, Damer, Dame, ,, Damerne, Kvinde, Hustru, Kone, kvindelig

1.8 ,, Moder, Mor, Damer, Dame, ., og, Damerne, Kvinde, Hustru

2.0 ,, Moder, Mor, ., Damer, og, Dame, at, en, i

freq ,, ., og, at, det, i, en, den, ikke, var



In [27]:
amtdot

,!,',(,),",",-,.,/,05,1,...,»,Â,Ã,â,˜,“,†,…,‰,€
!,2448.0,30.0,0.0,0.0,10459.0,1.0,8732.0,0.0,0.0,0.0,...,108.0,139.0,788.0,499.0,270.0,535.0,51.0,168.0,27.0,535.0
',30.0,27.0,0.0,0.0,260.0,0.0,133.0,0.0,0.0,0.0,...,9.0,15.0,15.0,7.0,4.0,7.0,1.0,3.0,0.0,7.0
(,0.0,0.0,23.0,23.0,85.0,3.0,48.0,4.0,0.0,0.0,...,3.0,2.0,9.0,1.0,3.0,1.0,0.0,1.0,0.0,1.0
),0.0,0.0,23.0,23.0,85.0,3.0,48.0,4.0,0.0,0.0,...,3.0,2.0,9.0,1.0,3.0,1.0,0.0,1.0,0.0,1.0
",",10459.0,260.0,85.0,85.0,178616.0,42.0,94849.0,2.0,0.0,4.0,...,1771.0,2098.0,13611.0,6146.0,4322.0,6245.0,985.0,2744.0,309.0,6245.0
-,1.0,0.0,3.0,3.0,42.0,22.0,27.0,1.0,4.0,0.0,...,1.0,1.0,6.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
.,8732.0,133.0,48.0,48.0,94849.0,27.0,97557.0,9.0,0.0,1.0,...,769.0,904.0,7622.0,3521.0,2654.0,3622.0,538.0,1389.0,213.0,3622.0
/,0.0,0.0,4.0,4.0,2.0,1.0,9.0,4.0,0.0,0.0,...,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
05,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
amt = paragraphs('ad3.txt')

In [14]:
pd.DataFrame(amt)

,0,1,2,3,4,5,6,7,8,9,...,555,556,557,558,559,560,561,562,563,564
0,Camilla,Collett,:,Amtmandens,DÃ,¸,tre,(,3.,utgave,...,None,None,None,None,None,None,None,None,None,None
1,Det,norske,sprÃ,¥,k,-,og,litteraturselskap,",",2013,...,None,None,None,None,None,None,None,None,None,None
2,Utgitt,i,samarbeid,med,Nasjonalbiblioteket,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,ISBN,:,978,-,82,-,93134,-,05,-,...,None,None,None,None,None,None,None,None,None,None
4,Teksten,er,lastet,ned,fra,bokselskap.no,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,Camilla,Collett,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,Amtmandens,DÃ,¸,tre,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,(,tredjeutgavens,versjon,",",fÃ,¸,rst,utgitt,1879,),...,None,None,None,None,None,None,None,None,None,None
8,2013,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
